In [2]:
# %connect_info
%qtconsole

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from graphgym.custom_graphgym.loader.protein import ProteinDataset
import matplotlib.pyplot as plt
import os

In [3]:
os.chdir('/Users/cgu3/Documents/Grape-Pi')

Import data from csv fies

In [15]:

# dataset = ProteinDataset("data/yeast-ORBI", numeric_columns=['protein probability'], label_column=None)
# dataset = ProteinBatchDataset("data/yeast-LCQ", numeric_params=['protein probability'])
# dataset = ProteinBatchDataset("data/yeast-ORBI-mRNA", numeric_params=['protein probability', 'mRNA(M)'])
# dataset = ProteinBatchDataset("data/yeast-LCQ-mRNA", numeric_params=['protein probability', 'mRNA(M)'])
# dataset = ProteinDataset("data/single-soft-label", numeric_columns=['protein_probability', 'mRNA_TPM'], label_column=None)
protein_dataset = ProteinDataset("data/single", numeric_columns=['protein_probability'], label_column=None)

In [16]:
protein_file = [f for f in os.listdir('data/single/raw/protein') if f.endswith('.csv')][0]
df = pd.read_csv(os.path.join('data/single/raw/protein', protein_file))
ids = df['protein.Accession'].values
# df = pd.read_csv('/Users/cgu3/Documents/data/SG_combined_protein_0982_with_protein_probability.csv')
df

,protein.Accession,protein_probability,protein_probability_soft_label,hard_label,gene_symbol,mRNA_TPM
0,A0A075B6H7,0.700122,0.267383,0,IGKV3-7,0.000000
1,A0A075B6H8,0.286947,0.056265,0,IGKV1D-42,0.000000
2,A0A075B6L6,0.943529,0.140940,0,TRBV7-3,0.000000
3,A0A075B6N1,0.289680,0.153119,0,TRBV19,0.000000
4,A0A075B6N2,0.259341,0.128836,0,TRBV20-1,0.370502
...,...,...,...,...,...,...
12532,Q9UI54,0.000000,0.000000,0,0,0.000000
12533,Q9Y3F1,0.000000,0.197423,0,0,0.000000
12534,Q9Y6C7,0.000000,0.093179,0,LINC00312,0.000000
12535,Q9Y6Z2,0.000000,0.000000,0,LINC01558,0.053100


In [17]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, labels, ids, train_mask=None, val_mask=None, test_mask=None):
        self.data = data
        self.labels = labels
        self.ids = ids

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample_data = self.data[index]
        sample_label = self.labels[index]
        sample_id = self.ids[index]
        return sample_data, sample_label, sample_id


In [18]:
from torch.utils.data import Dataset, Subset
dataset = CustomDataset(protein_dataset.x, protein_dataset.y, ids)

train_indices = torch.nonzero(protein_dataset.train_mask).squeeze().tolist()
train_dataset = Subset(dataset, train_indices)

val_indices = torch.nonzero(protein_dataset.val_mask).squeeze().tolist()
val_dataset = Subset(dataset, val_indices)

test_indices = torch.nonzero(protein_dataset.test_mask).squeeze().tolist()
test_dataset = Subset(dataset, test_indices)


In [25]:
train_dataset[1]

(tensor([0.9959, 3.9471]), tensor(1.), 'A0A075B6N1')

In [26]:
from torch.utils.data import DataLoader
torch.manual_seed(12345)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [27]:
for batch in train_dataloader:
    x, y, id = batch
    print(x.shape)
    print(y.shape)
    print(id)
    break

torch.Size([64, 2])
torch.Size([64])
('Q86TB9', 'P34995', 'P55265', 'Q5VSR9', 'Q86XA0', 'Q8IZF5', 'Q2M243', 'P20073', 'Q96BA8', 'P43681', 'P17858', 'Q8TDQ7', 'P50749', 'P54259', 'Q9BX73', 'Q96NK8', 'Q8IYI6', 'Q00839', 'Q13243', 'Q16880', 'Q92945', 'Q96PL1', 'Q8TEQ8', 'O14910', 'P49796', 'P0DME0', 'P13646', 'P05771', 'O60216', 'P56192', 'Q9BXL8', 'P17931', 'Q12972', 'Q8N3J5', 'Q15042', 'Q7Z7H3', 'Q68G75', 'Q8N2E6', 'Q96AE4', 'Q96AZ6', 'Q8TEV8', 'Q8IZM8', 'Q6PJT7', 'O60907', 'P00558', 'Q6IAN0', 'P31939', 'Q8NGB8', 'P00505', 'Q9BS26', 'Q8TF32', 'O75311', 'O75116', 'O15344', 'P01877', 'Q14739', 'Q5SY13', 'Q8IYK2', 'P18124', 'Q7KZF4', 'Q8IWX8', 'O15305', 'P23381', 'Q3KQU3')


In [28]:
import os
from torch import optim, nn, utils, Tensor
import pytorch_lightning as pl
from torchmetrics.classification import BinaryAUROC, BinaryAccuracy, BinaryF1Score
import torch


# define the LightningModule
class MLP(pl.LightningModule):
    def __init__(self, num_features, hidden_channels, num_classes, criterion):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = nn.Linear(num_features, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, num_classes)
        # self.lin3 = nn.Linear(hidden_channels, num_classes)
        self.criterion = criterion
        self.val_auroc = BinaryAUROC()
        self.test_auroc = BinaryAUROC()
        self.val_accuracy = BinaryAccuracy()
        self.test_accuracy = BinaryAccuracy()
        self.val_F1 = BinaryF1Score()
        self.test_F1 = BinaryF1Score()

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        return x

    def get_auc(self, out, target):
        return self.auroc(out, target)

    def training_step(self, batch, batch_idx):
        x, y, id = batch
        logits = self(x).squeeze(-1)  # Perform a single forward pass.

        loss = self.criterion(logits, y)  # Compute the loss solely based on the training nodes.
        values = {"loss": loss}
        self.log_dict(values, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y, ids = batch
        logits = self(x).squeeze(-1)  # Perform a single forward pass.
        self.val_accuracy.update(logits, y)
        loss = self.criterion(logits, y)  # Compute the loss solely based on the training nodes.
        self.val_auroc.update(logits, y)
        self.val_F1.update(logits, y)
        values = {"val_loss": loss, "val_acc": self.val_accuracy, "val_auroc": self.val_auroc, "val_F1": self.val_F1}
        self.log_dict(values, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y, ids = batch
        logits = self(x).squeeze(-1)
        self.test_accuracy.update(logits, y)
        loss = self.criterion(logits, y)
        self.test_auroc.update(logits, y)
        self.test_F1.update(logits, y)
        values = {"test_loss": loss, "test_acc": self.test_accuracy, "test_auroc": self.test_auroc, "test_F1": self.test_F1}
        self.log_dict(values, prog_bar=True, on_step=False, on_epoch=True)
        return loss
    
    def predict_step(self, batch, batch_idx):
        x, y, ids = batch
        ids = [item for sublist in ids for item in sublist]
        logits = self(x).squeeze(-1)
        pred_prob = torch.nn.functional.sigmoid(logits)
        return (ids, x[:, 0], x[:, 1], pred_prob, y)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer



In [29]:
# init the autoencoder
MLP_model = MLP(2, 10, 1, criterion = torch.nn.BCEWithLogitsLoss())
# MLP_model = MLP(dataset.num_node_features, 10, 2, criterion = torch.nn.CrossEntropyLoss())

In [30]:
import sys
from pytorch_lightning.callbacks import TQDMProgressBar


class MyProgressBar(TQDMProgressBar):
    def init_validation_tqdm(self):
        bar = super().init_validation_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    def init_predict_tqdm(self):
        bar = super().init_predict_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    def init_test_tqdm(self):
        bar = super().init_test_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar



In [34]:
import pytorch_lightning as pl
trainer = pl.Trainer(max_epochs=100, enable_progress_bar=False)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [35]:
trainer.fit(model=MLP_model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


  | Name          | Type              | Params
----------------------------------------------------
0 | lin1          | Linear            | 30    
1 | lin2          | Linear            | 11    
2 | criterion     | BCEWithLogitsLoss | 0     
3 | val_auroc     | BinaryAUROC       | 0     
4 | test_auroc    | BinaryAUROC       | 0     
5 | val_accuracy  | BinaryAccuracy    | 0     
6 | test_accuracy | BinaryAccuracy    | 0     
7 | val_F1        | BinaryF1Score     | 0     
8 | test_F1       | BinaryF1Score     | 0     
----------------------------------------------------
41        Trainable params
0         Non-trainable params
41        Total params
0.000     Total estimated model params size (MB)
/Users/cgu3/anaconda3/envs/grape-pi/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `Data

In [36]:
trainer.test(model=MLP_model, dataloaders=test_dataloader)

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test_F1            0.9259938597679138
        test_acc            0.8770325183868408
       test_auroc           0.9030500650405884
        test_loss           0.29189321398735046
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/Users/cgu3/anaconda3/envs/grape-pi/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[{'test_loss': 0.29189321398735046,
  'test_acc': 0.8770325183868408,
  'test_auroc': 0.9030500650405884,
  'test_F1': 0.9259938597679138}]

In [ ]:
out = trainer.predict(MLP_model, dataloaders=test_dataloader)
out

In [ ]:
accession, raw_prob, mRNA, pred_prob, soft_label = zip(*out)
accession = [item for sublist in accession for item in sublist]
raw_prob = [item.item() for sublist in raw_prob for item in sublist]
mRNA = [item.item() for sublist in mRNA for item in sublist]
pred_prob = [item.item() for sublist in pred_prob for item in sublist]
soft_label = [item.item() for sublist in soft_label for item in sublist]
pd.DataFrame({'accession': accession, 'raw_prob': raw_prob, 'mRNA': mRNA, 'pred_prob': pred_prob, 'soft_label': soft_label})

In [ ]:
test_dataloader

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/